# Как передаются параметры в функцию

В начале семинара 3 показаны отличительные особенности переменных в Ruby. Этот notebook идет немного дальше и показывет, что происходит с переменными при передаче в функцию.

Примеры иллюстрируют, что именно передается в функцию: копия переменной или ссылка на нее, а также в каких случая что происходит.

### Фабула

У нас есть объект - число 10 - который доступен глобально через переменную `x`. 

Графически это выглядит так:

![](img/fp/fp1.png)

Как вы помните, любая преременная в Ruby - это не переменная, а ссылка на объект, создаваемый интерпретатором.

Мы передаем `x` в функцию `foo`. 

Дальше нам нужно понять, передается ли в `arg` копия `x` или ссылка на нее. Если `arg` - это копия, то его можно менять без риска случайно испортить содержимое оригинальной переменной `x`. Если это ссылка, то любое изменение `arg` повлечет за собой изменение `x`. С точки зрения смысла в таком случае `x` и `arg` даже не имеет смысл их разделять, они ссылаются на одну и ту же переменную, созданную интерпретатором, только внутри функции мы обращаемся к ней через идентификатор `arg`, а снаружи через идентификатор `x`.

![](img/fp/fp2.png)

Как же нам понять, является ли `arg` ссылкой на `x` или ее копией? Есть достаточно простой способ сделать это. У каждого объекта в Ruby есть метод object_id, который возвращает число - уникальный номер объекта. По этому номеру можно однозначно идентифицировать объект. Если мы вернем object_id формального параметра `arg` и сравним его с object_id нашей переменной `x`, то мы поймем, тот же этот объект или нет. Здесь и далее для упрощения кода будем выводить результат операции сравнения. Если x.object_id **совпадает** с arg.object_id, то выводится **true**, в противном случае **false**.

Таким образом, если выводится true, то в функцию передается **ссылка** (то есть внутри функции foo мы работаем с той же переменной, что и снаружи).

In [6]:
def foo(arg)
  arg.object_id
end

x = 10
x.object_id == foo(x)

true

Тут передается **ссылка на х**.

### Когда создается новый объект

В следующем примере оператор `+=` не изменяет объект, переданный в качестве аргумента. Он создаёт новый объект.

Поэтому при сравнении object_id `x` и `arg` мы можем видеть их несовпадение. Как только мы пишем оператор `+=`, `arg` перестает ссылаться на ту же переменную, что и `x`. Интерпретатор создает новую переменную, копирует в нее содержимое переменной, на которую ссылалась переменная `arg`, прибавляет 1, после чего в `arg` записывается ссылка на новую переменную.

![](img/fp/fp3.png)

In [7]:
def foo1(arg)
  arg += 1
  return arg.object_id
end

x = 10
x.object_id == foo1(x)

false

Это работает не только с числом, но и со составными типами вроде массива (Array).

In [8]:
def foo2(arg)
  arg += [1]
  return arg.object_id
end

a = [1, 2, 3]
a.object_id == foo2(a)

false

### Когда изменяется текущий объект

Оператор `<<` изменяет объект, переданный в качестве аргумента.

Из функции возвращается ссылка на тот же самый объект, но модифицированный.

In [9]:
def foo2(arg)
  arg << 4
  return arg.object_id
end

a = [1, 2, 3]
a.object_id == foo2(a)

true

Мы никуда не записали результат функции `foo2`, но если мы посмотрим, что именно сейчас хранится в переменной `a`, то увидим, что это уже не `[1, 2, 3]`:

In [10]:
a

[1, 2, 3, 4]

### Итог

Параметры в функцию **всегда передаются по ссылке**. 

Если внутри функции вы используете методы, создающие новый объект (например, `+=`), то переданный объект не изменится. Если метод подразумевает изменение существующего (например, <<), то переданный объект изменится.

### И еще один пример...

То же самое можно проделать и для методов `map` (создает новый массив) и `map!` (изменяет существующий).

In [11]:
def foo_map(iterable)
  iterable.map {|x| x**2 }
end

def foo_map!(iterable)
  iterable.map! {|x| x**2 }
end

a = [1, 2, 3]
b = [1, 2, 3]

# Создалась копия
p a == foo_map(a)
# Изменился объект
p b == foo_map!(b)

false
true


true